In [1]:
from sympy import Tuple, Matrix

from sympde.core import Constant
from sympde.calculus import grad, dot, inner, cross, rot, curl, div
from sympde.calculus import laplace, hessian
from sympde.topology import (dx, dy, dz)
from sympde.topology import FunctionSpace, VectorFunctionSpace, Derham
from sympde.topology import element_of_space
from sympde.topology import ProductSpace
from sympde.topology import Boundary, NormalVector, TangentVector
from sympde.topology import Domain, Line, Square, Cube
from sympde.topology import Trace, trace_0, trace_1
from sympde.topology import Union
from sympde.expr import BilinearForm, LinearForm
from sympde.expr import Norm, TerminalExpr
from sympde.expr import find, EssentialBC


from psydac.fem.basic   import FemField
from psydac.fem.vector  import VectorFemField
from psydac.api.discretization import discretize
from psydac.fem.vector         import ProductFemSpace
from psydac.linalg.block       import BlockVector, BlockMatrix
from psydac.linalg.utilities   import array_to_stencil

from scipy.sparse.linalg import cg, gmres
from scipy.sparse.linalg import spsolve
from scipy import linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

import numpy as np
from sympy import lambdify

import sympy as sym

In [28]:
# ... create model with method of manufacture solutions
x, y, z = sym.symbols('x, y, z')

u = x*sym.sin(2*sym.pi*x)*y*(1 - sym.cos(2*sym.pi*y))*z*sym.sin(2*sym.pi*z)

f = -sym.diff(u, x, x) -sym.diff(u, y, y) - sym.diff(u, z, z)
#f = sym.lambdify((x, y, z), f)
u = sym.lambdify((x, y, z), u)
# ...

In [29]:
f

-4*pi*x*y*(pi*z*sin(2*pi*z) - cos(2*pi*z))*(cos(2*pi*y) - 1)*sin(2*pi*x) - 4*pi*x*z*(pi*y*cos(2*pi*y) + sin(2*pi*y))*sin(2*pi*x)*sin(2*pi*z) - 4*pi*y*z*(pi*x*sin(2*pi*x) - cos(2*pi*x))*(cos(2*pi*y) - 1)*sin(2*pi*z)

In [50]:
ncells = [5, 5, 5]
degree = [2, 2, 2]

# ... abstract model
domain = Cube()

derham = Derham(domain)
V0, V1, V2, V3 = derham.spaces

F = element_of_space(V2, name='F')

p, q = [element_of_space(V1, name=i) for i in ['p', 'q']]
u, v = [element_of_space(V0, name=i) for i in ['u', 'v']]

a  = BilinearForm(((p,u),(q,v)), dot(p,q) + dot(grad(u),q) - dot(p,grad(v)))
l = LinearForm((q,v), f*v)

domain_h = discretize(domain, ncells=ncells)
derham_h = discretize(derham, domain_h, degree=degree)

V0_h, V1_h, V2_h, V3_h = derham_h.spaces

Xh = V1_h * V0_h

ah = discretize(a, domain_h, [Xh, Xh])
lh = discretize(l, domain_h, Xh)

GRAD = derham_h.derivatives_as_matrices

In [51]:
L = lh.assemble()

In [58]:
L.blocks[3].toarray()

array([-1.99652402e-06, -1.55079392e-05, -3.82285592e-05,  1.10337913e-05,
        1.00565401e-04,  7.02714460e-05,  1.57123736e-05, -3.23847128e-05,
       -1.96687397e-04, -4.54520402e-04,  1.55195568e-04,  1.22040783e-03,
        8.38475180e-04,  1.81321507e-04, -1.91515351e-04, -1.06491169e-03,
       -2.39140403e-03,  8.75203365e-04,  6.48145177e-03,  4.41881477e-03,
        9.40583584e-04, -3.95858253e-04, -2.13570580e-03, -4.74547000e-03,
        1.78066076e-03,  1.29069448e-02,  8.77408653e-03,  1.85643576e-03,
       -3.59896774e-04, -1.94143816e-03, -4.31361382e-03,  1.61878909e-03,
        1.17325434e-02,  7.97563237e-03,  1.68745279e-03, -1.13036703e-04,
       -6.37863233e-04, -1.43961353e-03,  5.20608107e-04,  3.89535407e-03,
        2.65932807e-03,  5.67658585e-04, -1.51735289e-05, -1.09378375e-04,
       -2.64938612e-04,  8.01802595e-05,  7.00780303e-04,  4.87468738e-04,
        1.08043773e-04, -1.55079392e-05, -1.03689210e-04, -2.46332429e-04,
        7.84365804e-05,  

In [20]:
rhs.blocks

(<psydac.linalg.stencil.StencilVector at 0x7fb9445c08d0>,
 <psydac.linalg.stencil.StencilVector at 0x7fb9445c0940>)

In [18]:
M[0][0].toarray().shape

(294, 294)